<h1 style="text-align: center;">Project No. 1 - N-Grams & Sentiment Analysis</h1>

<h4 style="text-align: center;">

Esteban Gomez Valerio

Roi Jared Flores Garza Stone

Rafael Takata Garcia

Text Mining - O2025_MAF3654H

Ing. Juan Antonio Vega Fernández, M. Sc., M. T. Ed

ITESO
</h4>

#### Libraries

In [9]:
import torch
from torchtext.datasets import IMDB

OSError: [WinError 127] No se encontró el proceso especificado

#### Dataset

In [10]:
IMDB = torchtext.data

NameError: name 'torchtext' is not defined

In [ ]:
train_iter, test_iter = IMDB(root=".data")

for label, line in train_iter:
    print(f"Etiqueta: {label}, Texto: {line[:200]}...")
    break
